In [1]:
import os
print(os.getcwd())
current_path = os.getcwd()
current_path  = os.path.dirname(current_path)
current_path = os.path.dirname(current_path)
parent_path = os.path.dirname(current_path)
os.chdir(parent_path)
print(os.getcwd())

C:\Users\theal\PycharmProjects\ensembleLegends\src\deepReplication\deepBollinger
C:\Users\theal\PycharmProjects\ensembleLegends


In [157]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.metrics import Precision, Recall
from sklearn.metrics import confusion_matrix
from src.config import DEEP_DATA_PATH
import datetime
from src.deepReplication.dataPreperation.dataCombining import combine_data
from src.deepReplication.dataPreperation.scaling import scaling

def specificity(y_true, y_pred, threshold=0.5):
    y_pred_thresholded = tf.cast(y_pred > threshold, tf.bool)
    y_true = tf.cast(y_true, tf.bool)

    true_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.logical_not(y_true), tf.logical_not(y_pred_thresholded)), dtype=tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.logical_not(y_true), y_pred_thresholded), dtype=tf.float32))

    return true_negatives / (true_negatives + false_positives + tf.keras.backend.epsilon())

def recall(y_true, y_pred, threshold=0.5):
    y_pred_thresholded = tf.cast(y_pred > threshold, tf.bool)
    y_true = tf.cast(y_true, tf.bool)

    true_positives = tf.reduce_sum(tf.cast(tf.logical_and(y_true, y_pred_thresholded), dtype=tf.float32))
    false_negatives = tf.reduce_sum(tf.cast(tf.logical_and(y_true, tf.logical_not(y_pred_thresholded)), dtype=tf.float32))

    return true_positives / (true_positives + false_negatives + tf.keras.backend.epsilon())


precison = Precision()

In [158]:
# Current time for unique folder names
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Subdirectory for the current training run
log_dir = "logs/fit/" + current_time  # E.g., logs/fit/20230315-123456

# Create a TensorBoard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [159]:
num_prev_prices = 20

running_data_preprocesing = "Box"
identifier = 'test1box'

split_to_do = "34_33_33"

In [160]:
# if running_data_preprocesing == "Bollinger":
#     combine_data(base_name='deepBollinger', identifier=identifier, num_prev_prices=num_prev_prices, drop_nans=True, splits=split_to_do)
#     # combine_data(base_name='deepBollinger', identifier='test1bollinger', num_prev_prices=20, drop_nans=True)
#     scaling(base_name='Bollinger', identifier=identifier, num_prev_price=num_prev_prices, splits=split_to_do)
#
# if running_data_preprocesing == "Turtle":
#     combine_data(base_name='deepTurtle', strategy='turtle_naive', identifier=identifier, num_prev_prices=num_prev_prices, drop_nans=True)
#     scaling(base_name='Turtle', identifier=identifier, num_prev_price=num_prev_prices)
#
# if running_data_preprocesing == "Box":
#     combine_data(base_name='deepBox', strategy='box_naive', identifier=identifier, num_prev_prices=num_prev_prices, drop_nans=True)
#     scaling(base_name='Box', identifier=identifier, num_prev_price=num_prev_prices)

In [161]:
path = os.path.join(DEEP_DATA_PATH, f'deep{running_data_preprocesing}', f'tst{split_to_do}_scaled_combined_{identifier}_20100104_to_20201231_doctest2_numP{num_prev_prices}.csv')
# path = os.path.join(DEEP_DATA_PATH, 'deepTurtle', 'scaled_combined_turtle_naive_20100104_to_20201231_doctest.csv')
# path = os.path.join(DEEP_DATA_PATH, 'deepBox', 'scaled_combined_box_naive_20100104_to_20201231_doctest.csv')

df = pd.read_csv(path, low_memory=False)

In [162]:
trade_counts = df['is_trade'].value_counts()
print(trade_counts)

0    22129
1    21966
Name: is_trade, dtype: int64


In [163]:
# Prepare input and target
X = df[[f'PrevPrice_{i}' for i in range(num_prev_prices + 1)]].values
y = df['is_trade'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)

series = pd.Series(y_test)
trade_counts = series.value_counts()
print(trade_counts)

0    4422
1    4397
dtype: int64


In [164]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', precison, specificity, recall])

In [165]:
model.fit(X_train, y_train, epochs=1, batch_size=32, validation_data=(X_test, y_test))

1103/1103 [==============================] - 12s 10ms/step - loss: 0.4299 - accuracy: 0.7937 - precision_7: 0.7687 - specificity: 0.7513 - recall: 0.8376 - val_loss: 0.3401 - val_accuracy: 0.8542 - val_precision_7: 0.8215 - val_specificity: 0.7862 - val_recall: 0.9025


In [166]:
results = model.evaluate(X_test, y_test)

276/276 [==============================] - 1s 3ms/step - loss: 0.3401 - accuracy: 0.8542 - precision_7: 0.8215 - specificity: 0.7862 - recall: 0.9025


In [167]:
print(results[1])

0.8541784882545471


In [168]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype("int32")
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[3558  864]
 [ 422 3975]]


In [169]:
# 1 if X_test[0] == 1 or X_test[0] == 0

def naive_model(X_test, y_test):
    preds = []
    for x in X_test:
        if float(x[0]) > .95 or float(x[0]) < .05:
            preds.append(1)
        else:
            preds.append(0)

    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()

    # Calculate metrics
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if tp + fp != 0 else 0
    specificity = tn / (tn + fp) if tn + fp != 0 else 0
    recall = tp / (tp + fn) if tp + fn != 0 else 0  # Added recall calculation

    return accuracy, precision, specificity, recall




In [170]:
acc, prec, spec, recall = naive_model(X_test, y_test)

In [171]:
print(acc, prec, spec, recall)

0.8009978455607212 0.7151465798045603 0.6044776119402985 0.9986354332499432


In [172]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import numpy as np

# Create a Gaussian Naive Bayes classifier
nb_model = GaussianNB()

# Train the model using the training sets
nb_model.fit(X_train, y_train)

# Predict on test set
y_pred = nb_model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)

# Precision
precision = precision_score(y_test, y_pred)

# Recall
recall = recall_score(y_test, y_pred)

# Specificity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)

# Displaying the metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall (Sensitivity): {recall}")
print(f"Specificity: {specificity}")

Accuracy: 0.8430661072683978
Precision: 0.7651821862348178
Recall (Sensitivity): 0.9886286104161929
Specificity: 0.6983265490728178


In [173]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, log_loss
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

# Create polynomial features of degree 2
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Create a logistic regression classifier
log_reg = LogisticRegression(max_iter=1000)

# Train the model using the training sets with polynomial features
log_reg.fit(X_train_poly, y_train)

# Predict on test set with polynomial features
y_pred = log_reg.predict(X_test_poly)
y_pred_proba = log_reg.predict_proba(X_test_poly)[:, 1]  # Probabilities for the positive class

# Binary Cross-Entropy Loss
loss = log_loss(y_test, y_pred_proba)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)

# Precision
precision = precision_score(y_test, y_pred)

# Recall
recall = recall_score(y_test, y_pred)

# Specificity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)

# Displaying the metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall (Sensitivity): {recall}")
print(f"Specificity: {specificity}")

Accuracy: 0.8843406281891371
Precision: 0.8400805639476334
Recall (Sensitivity): 0.9486013190811917
Specificity: 0.8204432383536862


In [174]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Create a K-NN classifier. You can change 'n_neighbors' to optimize performance
knn = KNeighborsClassifier(n_neighbors=5)

# Train the model using the training sets
knn.fit(X_train, y_train)

# Predict on test set
y_pred = knn.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)

# Precision
precision = precision_score(y_test, y_pred)

# Recall
recall = recall_score(y_test, y_pred)

# Specificity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)

# Displaying the metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall (Sensitivity): {recall}")
print(f"Specificity: {specificity}")

Accuracy: 0.7779793627395396
Precision: 0.7318000380155864
Recall (Sensitivity): 0.8755969979531498
Specificity: 0.6809136137494346


In [175]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Create a Random Forest classifier
rf = RandomForestClassifier(n_estimators=100)  # n_estimators is the number of trees in the forest

# Train the model using the training sets
rf.fit(X_train, y_train)

# Predict on the test set
y_pred = rf.predict(X_test)

# Evaluate the model
# Accuracy
accuracy = accuracy_score(y_test, y_pred)

# Precision
precision = precision_score(y_test, y_pred)

# Recall
recall = recall_score(y_test, y_pred)

# Specificity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)

# Displaying the metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall (Sensitivity): {recall}")
print(f"Specificity: {specificity}")

Accuracy: 0.9023698832067127
Precision: 0.8646864686468647
Recall (Sensitivity): 0.9533773027063908
Specificity: 0.8516508367254636
